<a href="https://colab.research.google.com/github/bhattacharjee/Capsule-Network-Keras/blob/master/modified_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!rm -rf *
!git clone https://github.com/TheLastFrame/Capsule-Network-Keras.git
!cp -r Capsule-Network-Keras/* .


Cloning into 'Capsule-Network-Keras'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 133 (delta 46), reused 34 (delta 12), pack-reused 45
Receiving objects: 100% (133/133), 36.79 KiB | 1.11 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [9]:
from keras.datasets import mnist
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [10]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [13]:
from capsulelayer import Capsule
class CapsuleModel:
    @staticmethod
    def build(train_X, train_y, test_X, test_y):
        classes = np.unique(train_y).shape[0]
        routing=5
        capsule_dim=16
        input_image = tf.keras.Input(shape=train_X[0].shape)
        capsule = Capsule(classes, capsule_dim, routing, True)(input_image) #num capsule (classes), dim capsule, routings
        output = tf.keras.layers.Lambda(lambda x: tf.math.sqrt(tf.reduce_sum(tf.math.square(x), 2)), output_shape=(classes,))(capsule)

        model = tf.keras.Model(inputs=input_image, outputs=output)
        model.compile(loss=tf.losses.categorical_crossentropy,
                    optimizer='adam',
                    metrics=['accuracy'])
        return model


In [15]:
print(np.unique(train_y))
m = CapsuleModel.build(train_X, train_y, test_X, test_y)
print(tf.one_hot(train_y, depth=10).numpy().shape, train_X.shape)
m.fit(x=train_X, y=tf.one_hot(train_y, depth=10), epochs=2)
print(m)

[0 1 2 3 4 5 6 7 8 9]
(60000, 10) (60000, 28, 28)
Epoch 1/2
1875/1875 [==============================] - 8s 4ms/step - loss: nan - accuracy: 0.0987
Epoch 2/2
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.0987


In [19]:
tf.keras.models.save_model(m, 'save.h5')
m.save("./save.tf", save_format='tf')

INFO:tensorflow:Assets written to: ./save.tf/assets


In [21]:
!ls save.tf


assets	keras_metadata.pb  saved_model.pb  variables


In [24]:
run_model = tf.function(lambda x : m(x))
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(m.inputs[0].shape, m.inputs[0].dtype)
)


In [27]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

with open("./lite" + "_conc_func"+ '.tflite', 'wb') as f:
    f.write(tflite_model)

converter = tf.lite.TFLiteConverter.from_keras_model(m)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

with open("./lit2" + '.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpraz4ttrj/assets


INFO:tensorflow:Assets written to: /tmp/tmpraz4ttrj/assets
